# Import Libraries

In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import Adam

In [18]:
import os

In [19]:
print(os.getcwd())

/Users/javi/Desktop/school/CSE 120/Anamoly Detection/Leaf-Classification/data/tomato


In [20]:
train_data_dir = 'train'

# Check if the directory exists
if not os.path.exists(train_data_dir):
    print(f"The directory {train_data_dir} does not exist.")
else:
    print(f"Found directory {train_data_dir}. Proceeding with loading data...")
    # Proceed with your data loading and model training


Found directory train. Proceeding with loading data...


# Data loading and preprocessing

In [21]:
train_data_dir = 'train'
test_data_dir = 'val'

In [22]:
train_datagen = ImageDataGenerator(
    rescale=1/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.3
)

In [23]:
train_gen = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 7000 images belonging to 10 classes.


In [24]:
val_gen = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 3000 images belonging to 10 classes.


In [25]:
test_datagen = ImageDataGenerator(rescale=1/255.0)

In [26]:
test_gen = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 1000 images belonging to 10 classes.


# Model architecture

In [27]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')  # Assuming we have 10 classes, adjust if necessary
])

# Compile the model

In [28]:
model.compile(
    optimizer=Adam(learning_rate=0.0001), # add learning rate (lr=0.0001) default was 0.001
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Early Stopping to avoid overfitting

In [29]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=20,
    verbose=1,
    restore_best_weights=True
)

# Train the model

In [30]:
history = model.fit(
    train_gen,
    epochs=80,
    validation_data=val_gen,
    callbacks=[early_stopping]
)

Epoch 1/80


2024-04-01 15:58:52.154448: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


178/219 [=======================>......] - ETA: 20s - loss: 2.2785 - accuracy: 0.1129

KeyboardInterrupt: 


# Evaluate the model

In [31]:
scores = model.evaluate(test_gen) #change test_gen to val_gen
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

32/32 [==============================] - 4s 133ms/step - loss: 2.2085 - accuracy: 0.1950
Test loss: 2.2084712982177734
Test accuracy: 0.19499999284744263


# Save the model

In [32]:
model.save('merged_model.h5')